In [125]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from datetime import timedelta
from catboost import CatBoostRegressor
import pickle
import os

Ранее мы попробывали различные способы предсказания. Разделим наши товары на 3 категории:

1) товары у которых не сильно меняется цена и лучше всего сработает простое предсказание по последней цене с опредленным диапозоном.

2) товары на которых лучшее качество показала модель catboosta + товары с наибольшей корреляцией к биржевым котировкам. Для них обучим модели с полным сетом признаков.

3) товары которые не предсказываются. Для них не будем выдавать предсказание

In [168]:
top = pd.read_excel('baseline_top.xlsx')
top

,товары,median_delviery_time,plan_delivery_time,n_rows,mean_cost,std_cost,train_sample,precentege_std,baseline_mean_mape,baseline_std_mape,linear_mean_mape,linear_std_mape,catboost_mean_mape,catboost_std_mape,today_prediction,last_price
0,Зуб 1085.52.05-1,69.0,75,73,9134.237239,602.284793,54,0.065937,0.000000,0.000000,0.0327,0.0210,0.061089,0.071138,9224.041862,9388.535032
1,Колесо 3519.05.02.006,139.0,220,54,365130.478887,109083.122734,39,0.298751,12.640331,14.447244,143.8520,360.9490,6.550194,7.241393,393392.436986,423566.878981
2,Вал 3536.11.01.002,124.0,280,32,233428.542994,53077.194660,24,0.227381,24.954722,14.950492,14.6251,13.9510,7.935902,6.430322,295486.104537,321656.050955
3,Вал 3519.05.02.083,205.0,330,26,265467.246448,33446.110431,18,0.125990,3.562355,3.917742,7.3704,4.6415,4.048922,3.752559,NaN,315286.624204
4,Ось 3519.21.00.025,107.0,180,22,18491.603937,8438.317460,15,0.456332,38.293052,55.099867,51.4163,49.1862,10.947940,4.348130,18442.177219,18280.254777
5,Коромысло 3519.21.00.023,108.0,220,18,167182.014154,27669.278965,12,0.165504,16.050988,14.419305,31.0658,54.4411,5.411067,5.424545,162259.285106,185900.458599
6,Вал 3572.05.10.200,224.0,220,17,254809.741476,13064.105603,12,0.051270,4.612192,6.043061,4.0499,6.5345,5.999284,5.089663,NaN,213375.796178
7,Вант стрелы 00.1606.49.1,72.0,70,24,388241.454352,64302.142058,18,0.165624,14.004953,17.348959,21.0059,4.9300,22.188507,10.019942,NaN,465688.535032
8,Засов 3532.01.02.007,47.0,60,24,13896.199310,1993.187524,18,0.143434,9.918709,10.015745,5.8587,6.3289,11.217132,4.090416,NaN,16401.273885
9,Вал 3572.05.10.150,106.5,220,22,211202.652866,46355.784495,15,0.219485,3.884510,6.045484,5.1259,2.9812,3.405469,3.238360,NaN,219745.222930


In [162]:
#basline_mape <10% и низкая дисперсия в цене
simple_category = ['Зуб 1085.52.05-1', 'Вал 3519.05.02.083','Вал 3572.05.10.200','Засов 3532.01.02.007',
                  'Засов 3532.01.02.007', 'Вал 3572.05.10.150', 'Сателлит 3536.11.01.028', 'Колесо 3519.05.02.061',
                  'Вал 3519.05.02.007', 'Ось 3536.03.00.001', 'Коромысло 3537.25.00.500 ']

#catboost_model < 10%
model_category = ['Колесо 3519.05.02.006','Вал 3536.11.01.002','Ось 3519.21.00.025','Коромысло 3519.21.00.023',
                 'ВАЛ-ШЕСТЕРНЯ 3536.11.01.002 ','Вал 3532.10.01.011', 'Колесо 3519.05.02.003',
                 'Ось 3519.05.02.081']
#высокая дисперсия и ошибка более 10% везде
unreal_category = ['Вант стрелы 00.1606.49.1 ','Ось 3519.05.02.082', ]

## Предобработка файлов

In [24]:
df = pd.read_csv('datamon_with_indexes.csv', index_col=0)
df['Дата поставки'] = pd.to_datetime(df['Дата поставки'])
df['Дата заказа'] = pd.to_datetime(df['Дата заказа'])
print(df.shape)


(3164, 297)
(21, 1)


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3164 entries, 0 to 2955
Columns: 297 entries, Наименование to RUB=X
dtypes: datetime64[ns](2), float64(290), int64(2), object(3)
memory usage: 7.2+ MB


In [26]:
#метрика MAPE процентная ошибка
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [27]:
df['date'] = (pd.to_datetime(df['Дата заказа']) - pd.to_datetime('2015-06-09')).dt.days

## Catboost с дополнительными внутренними фичами

Создаем признаки

In [28]:
#фичи по товарам
for name in df['Наименование'].unique():
    t = df[df['Наименование']==name]
    for i in t.index:
        df.loc[i, 'previous_mean'] = t.iloc[:i]['Цена, руб'].mean()
        df.loc[i, 'previous_max'] = t.iloc[:i]['Цена, руб'].max()
        df.loc[i, 'previous_min'] = t.iloc[:i]['Цена, руб'].min()
        df.loc[i, 'previous_std'] = t.iloc[:i]['Цена, руб'].std()
   
        
        df.loc[i, 'cost_previous_30d'] = t[(t['date']< t.loc[i]['date']) &(t['date']>= t.loc[i]['date']-30)]['Цена, руб'].mean()
        df.loc[i, 'cost_previous_100d'] = t[(t['date']< t.loc[i]['date']) &(t['date']>= t.loc[i]['date']-100)]['Цена, руб'].mean()
        df.loc[i, 'cost_previous_300d'] = t[(t['date']< t.loc[i]['date']) &(t['date']>= t.loc[i]['date']-300)]['Цена, руб'].mean()
        
        try:
            df.loc[i, 'previous_cost'] = t.iloc[previous_index]['Цена, руб']
        except:
            df.loc[i, 'previous_cost'] = np.nan
            
        previous_index = i
        
#фичи по поставщикам
for name in df['Поставщик'].unique():
    t = df[df['Поставщик']==name]
    for i in t.index:
        df.loc[i, 'previous_deliver_difference'] = t.iloc[:i]['Наименование'].nunique()
        df.loc[i, 'previous_deliver_mean'] = t.iloc[:i]['Цена, руб'].mean()
        df.loc[i, 'previous_deliver_max'] = t.iloc[:i]['Цена, руб'].max()
        df.loc[i, 'previous_deliver_min'] = t.iloc[:i]['Цена, руб'].min()
        df.loc[i, 'previous_deliver_std'] = t.iloc[:i]['Цена, руб'].std()
        
        
        df.loc[i, 'deliver_previous_30d'] = t[(t['date']< t.loc[i]['date']) &(t['date']>= t.loc[i]['date']-30)]['Цена, руб'].mean()
        df.loc[i, 'deliver_previous_100d'] = t[(t['date']< t.loc[i]['date']) &(t['date']>= t.loc[i]['date']-100)]['Цена, руб'].mean()
        df.loc[i, 'deliver_previous_300d'] = t[(t['date']< t.loc[i]['date']) &(t['date']>= t.loc[i]['date']-300)]['Цена, руб'].mean()
        



In [49]:
def create_features_from_indexes(df, column_name):
    for name in df['Наименование'].unique():
        t = df[df['Наименование']==name]
        try:
            index_table = pd.read_csv('parser/basic_matrials_load/'+column_name+'.csv', index_col=0)
        except:
            index_table = pd.read_csv('parser/basic_futures_and_other_load/'+column_name+'.csv', index_col=0)
            
        index_table.index = pd.to_datetime(index_table.index)
        for i in t.index:
            df.loc[i, '{}_max_last_30'.format(column_name)] = index_table[(index_table.index< df.loc[i]['Дата заказа']) 
                                                                   &(index_table.index> df.loc[i]['Дата заказа'] -pd.to_timedelta(30, unit='d'))][column_name].max()
            df.loc[i, '{}_mean_last_30'.format(column_name)] = index_table[(index_table.index< df.loc[i]['Дата заказа']) 
                                                                   &(index_table.index> df.loc[i]['Дата заказа'] -pd.to_timedelta(30, unit='d'))][column_name].mean() 
            df.loc[i, '{}_min_last_30'.format(column_name)] = index_table[(index_table.index< df.loc[i]['Дата заказа']) 
                                                                   &(index_table.index> df.loc[i]['Дата заказа'] -pd.to_timedelta(30, unit='d'))][column_name].min()
            
            df.loc[i, '{}_max_last_100'.format(column_name)] = index_table[(index_table.index< df.loc[i]['Дата заказа']) 
                                                                   &(index_table.index> df.loc[i]['Дата заказа'] -pd.to_timedelta(100, unit='d'))][column_name].max()
            df.loc[i, '{}_mean_last_100'.format(column_name)] = index_table[(index_table.index< df.loc[i]['Дата заказа']) 
                                                                   &(index_table.index> df.loc[i]['Дата заказа'] -pd.to_timedelta(100, unit='d'))][column_name].mean() 
            df.loc[i, '{}_min_last_100'.format(column_name)] = index_table[(index_table.index< df.loc[i]['Дата заказа']) 
                                                                   &(index_table.index> df.loc[i]['Дата заказа'] -pd.to_timedelta(100, unit='d'))][column_name].min()
            
            df.loc[i, '{}_max_last_360'.format(column_name)] = index_table[(index_table.index< df.loc[i]['Дата заказа']) 
                                                                   &(index_table.index> df.loc[i]['Дата заказа'] -pd.to_timedelta(360, unit='d'))][column_name].max()
            df.loc[i, '{}_mean_last_360'.format(column_name)] = index_table[(index_table.index< df.loc[i]['Дата заказа']) 
                                                                   &(index_table.index> df.loc[i]['Дата заказа'] -pd.to_timedelta(360, unit='d'))][column_name].mean() 
            df.loc[i, '{}_min_last_360'.format(column_name)] = index_table[(index_table.index< df.loc[i]['Дата заказа']) 
                                                                   &(index_table.index> df.loc[i]['Дата заказа'] -pd.to_timedelta(360, unit='d'))][column_name].min()
            
    return df
                
            
    

In [59]:
for column in list(df.columns)[11:297]:
    df = create_features_from_indexes(df, column)
    print(column)

IBRIGST.BO
BHARATWIRE.BO
APCL.BO
SARDAEN.BO
NEXA
CENTEXT.BO
LAC
INVPRECQ.BO
BEEKAY.BO
HINDALUMI.BO
RATHIBAR.BO
VASWANI.BO
TINPLATE.BO
TNSTLTU.BO
MSPL.BO
CRH
MDRNSTL.BO
VALE
BIRLACORPN.BO
STARLIT.BO
BCC
VEDL.BO
ROHITFERRO.BO
FACORALL.BO
ASHAPURMIN.BO
ACC.BO
GOPAIST.BO
AML.BO
SAGCEM.BO
NCLIND.BO
BHP
BAROEXT.BO
ISMTLTD.BO
BVN
TMST
UMIYA.BO
KAMDHENU.BO
INDSILHYD.BO
USAU
SA
METC
SHREMETAL.BO
VSSL.BO
BEDMUTHA.BO
RAJGLOWIR.BO
CPAC
INDIACEM.BO
PAAS
UNIABEXAL.BO
JSL.BO
IFGLEXPOR.BO
HL-PB
UTTAMSTL.BO
KRIINFRA.BO
HISARMET.BO
ANKITMETAL.BO
MANAKSTELTD.BO
SATINDLTD.BO
NORTHLINK.BO
SANDUMA.BO
USHAMART.BO
RISHDIGA.BO
NMG
METALCO.BO
CREATIVE.BO
JINDALSTEL.BO
INDINFRA.BO
GMETCOAL.BO
RAMCOCEM.BO
ASHSI.BO
GFI
USLM
SOUTHMG.BO
SYNL
SAURASHCEM.BO
JKLAKSHMI.BO
GPIL.BO
SHREDIGCEM.BO
FCX
APLAPOLLO.BO
NOVIS.BO
JTLINFRA.BO
JAYNECOIND.BO
SUM
MANGLMCEM.BO
UNITINT.BO
SHAHALLOYS.BO
KEERTHI.BO
OR
LGO
SICAGEN.BO
SCFL.BO
ZEUS
BVCL.BO
LITM
SHRDAIS.BO
ORISSAMINE.BO
STDBAT.BO
CLF
HINDZINC.BO
MAITHANALL.BO
AEM
ANDHRACEMT.B

In [62]:
df.to_csv('datamon_prepared.csv', index= False)

In [80]:
cat_features = ['Регион', 'Условия платежа', 'НРП - нерегламентная потребность (внеплановая закупка)', 'Поставщик' ]
num_features = list(df.columns)
for col in cat_features:
    num_features.remove(col)
num_features .remove('Наименование')
num_features .remove('Цена, руб')
num_features .remove('Дата поставки')
num_features .remove('Дата заказа')


for feature in cat_features:
    df[feature]=df[feature].astype(str)

In [138]:
for name in model_category:

    if name +'.pickle' in os.listdir('models/'):
        continue
    t = df[df['Наименование']==name]
    print('Тренируем модель', name)
    train = t[list(cat_features)+num_features+['Цена, руб']]
    model = CatBoostRegressor(iterations=1000,
                                random_seed=63,
                                learning_rate=0.05,
                                early_stopping_rounds= 43,
                                thread_count= 35,
                                cat_features = cat_features,
                                silent=True).fit(train.drop('Цена, руб',axis=1), train['Цена, руб'])
            
    #сразу сделаем предсказание на сегодняшний день и сохраним в топ
    
    today = pd.DataFrame(t.iloc[-1]).T
    today['Дата заказа'] =pd.to_datetime('17.03.2022')
   

    for column in list(df.columns)[11:297]:
        today = create_features_from_indexes(today, column)
        
    prediction = model.predict(np.array(today[cat_features+num_features]).reshape(1, -1))
   
    top.loc[top['товары']==name,'today_prediction'] = prediction
    pickle.dump(model, open('models/'+name+'.pickle', 'wb'))

    

Тренируем модель Зуб 1085.52.05-1


/home/jupyteruser/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


Тренируем модель Колесо 3519.05.02.003
Тренируем модель Ось 3519.05.02.081


In [164]:
for name in top['товары']:
    top.loc[top['товары']==name,'last_price'] = df[df['Наименование']==name]['Цена, руб'].iloc[-1]
    

In [169]:
top

,товары,median_delviery_time,plan_delivery_time,n_rows,mean_cost,std_cost,train_sample,precentege_std,baseline_mean_mape,baseline_std_mape,linear_mean_mape,linear_std_mape,catboost_mean_mape,catboost_std_mape,today_prediction,last_price
0,Зуб 1085.52.05-1,69.0,75,73,9134.237239,602.284793,54,0.065937,0.000000,0.000000,0.0327,0.0210,0.061089,0.071138,9224.041862,9388.535032
1,Колесо 3519.05.02.006,139.0,220,54,365130.478887,109083.122734,39,0.298751,12.640331,14.447244,143.8520,360.9490,6.550194,7.241393,393392.436986,423566.878981
2,Вал 3536.11.01.002,124.0,280,32,233428.542994,53077.194660,24,0.227381,24.954722,14.950492,14.6251,13.9510,7.935902,6.430322,295486.104537,321656.050955
3,Вал 3519.05.02.083,205.0,330,26,265467.246448,33446.110431,18,0.125990,3.562355,3.917742,7.3704,4.6415,4.048922,3.752559,NaN,315286.624204
4,Ось 3519.21.00.025,107.0,180,22,18491.603937,8438.317460,15,0.456332,38.293052,55.099867,51.4163,49.1862,10.947940,4.348130,18442.177219,18280.254777
5,Коромысло 3519.21.00.023,108.0,220,18,167182.014154,27669.278965,12,0.165504,16.050988,14.419305,31.0658,54.4411,5.411067,5.424545,162259.285106,185900.458599
6,Вал 3572.05.10.200,224.0,220,17,254809.741476,13064.105603,12,0.051270,4.612192,6.043061,4.0499,6.5345,5.999284,5.089663,NaN,213375.796178
7,Вант стрелы 00.1606.49.1,72.0,70,24,388241.454352,64302.142058,18,0.165624,14.004953,17.348959,21.0059,4.9300,22.188507,10.019942,NaN,465688.535032
8,Засов 3532.01.02.007,47.0,60,24,13896.199310,1993.187524,18,0.143434,9.918709,10.015745,5.8587,6.3289,11.217132,4.090416,NaN,16401.273885
9,Вал 3572.05.10.150,106.5,220,22,211202.652866,46355.784495,15,0.219485,3.884510,6.045484,5.1259,2.9812,3.405469,3.238360,NaN,219745.222930


In [171]:
top.to_csv('top_final.csv', index= False)